# LAB GenAI - LLMs - OpenAI Assistant

## Exercise 1

Create an assistant to answer a topic of your choosing:
 - Upload a file of your interest
 - Add Instructions to the prompt
 - Use the assistant in Playground mode

 https://platform.openai.com/playground/assistants

In [88]:
import openai
import pandas as pd

#OpenAI API key
api_key = ''

## Create the assistant
from openai import OpenAI
 
client = OpenAI(api_key=api_key)
model_params = {'model': 'gpt-4o-mini', 'temperature': 0.4, 'max_tokens': 200}
assistant = client.beta.assistants.create(
  name="FAQs Helper",
  instructions="You are an expert in answering questions",
  model="gpt-4o-mini")

In [78]:
## OPEN AI EMBEDDINGS:
from langchain_openai import OpenAIEmbeddings
import os
# import
from langchain.vectorstores import Chroma
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)
from langchain_text_splitters import CharacterTextSplitter

API_KEY = ""

# Create the embeddings function
embeddings = OpenAIEmbeddings(model="text-embedding-3-small", api_key = API_KEY)

# create a text splitter
text_splitter = CharacterTextSplitter(chunk_size=100, chunk_overlap=0)
print('Cell finished')

Cell finished


In [79]:
# load the document and split it into chunks
document_dir = "./"
filename = "FAQ.pdf"
file_path = os.path.join(document_dir, filename)

pages = PyPDFLoader(file_path).load_and_split()
docs = text_splitter.split_documents(pages)

# load it into Chroma
db = Chroma.from_documents(docs, embeddings, persist_directory="./chroma_db")
print('Cell finished')

Cell finished


In [80]:
user_question = "How can I apply for UNDP?"
docs = db.similarity_search(user_question, k=10)

# print results
for doc in docs[0:3]:
    print(doc.page_content, '\n')
    print()

Number of requested results 10 is greater than number of elements in index 7, updating n_results = 7


Frequently Asked Questions (FAQs)   
6 
 FF FFREQUENTLY REQUENTLY REQUENTLY REQUENTLY AA AASKED SKED SKED SKED QQ QQUESTIONS UESTIONS UESTIONS UESTIONS (FAQ (FAQ (FAQ (FAQ SS SS)) ))    
     UNDP’s eRecruit System  
 
4.7 
 
What if I want to submit additional documents to my application? 
UNDP does not require that you attach additional documents to your application. 
However, at any time during the recruitment process, you may be requested by a 
UNDP Recruitment Specialist to submit additional validating documents to your 
personal profile in support of your application. If you receive a request to upload 
specific documents, first navigate to the Careers Home page /barb2right/barb2right /barb2right/barb2right My Career Tools 
page . Click on Applicant Attachments and add the relevant documents in the 
attachments section. 
 
4. 8 Can I apply for more than one job posting at the same time ? 
You may apply to multiple UNDP job postings given that you meet the minimum 
requirements fo

In [82]:
def _get_document_prompt(docs):
    prompt = '\n'
    for doc in docs:
        prompt += '\nContent:\n'
        prompt += doc.page_content + '\n\n'
    return prompt

In [83]:
prompt = f"""
## INTRODUCTION
You are a Chatbot designed to help answer technical questions about a eRecuiiting process.
The user asked: "{user_question}"

## CONTEXT
Technical Documentation for the recruitment process:
'''
{_get_document_prompt(docs)}
'''

## RESTRICTIONS
Refer to the products by their names.
Be clear, transparent, and factual: only state what is in the context without providing opinions or subjectivity.
Answer the question based solely on the context above; if you do not know the answer, be clear with the user that you do not know.
Only respond to questions related to the products, avoiding jokes, offensive remarks, and discussions on religion or sexuality.
If the user does not provide sufficient context, do not answer and instead ask for more information on what the user wants to know.

## TASK
First, answer directly to the user, if possible
Second, point the user int he right direction of the documentation
Lastly, answer in Markdown format

## RESPONSE STRUCTURE:
'''
# [Answer Title]
[answer text]

Source:
• From pages [...] of the Technical Documentation for *Product1* (link)
• From pages [...] of the Technical Documentation for *Product2* (link)
'''

## CONVERSATION:
User: {user_question}
Agent:
"""

In [89]:
thread = client.beta.threads.create(
  messages=[
    {
      "role": "user",
      "content": prompt,
    }
  ]
)

run = client.beta.threads.runs.create_and_poll(
    thread_id=thread.id, assistant_id=assistant.id
)

messages = list(client.beta.threads.messages.list(thread_id=thread.id, run_id=run.id))

message_content = messages[0].content[0].text
print(message_content.value)

# How to Apply for UNDP
To apply for a job with UNDP, you must use the UNDP eRecruit system. Here are the steps you need to follow:

1. **Register**: Create a personal profile in the UNDP eRecruit system.
2. **View Job Descriptions**: Browse through the available job postings and select the ones you are interested in.
3. **Complete Application**: Fill out all required sections of your application, including personal information, job requirements, motivation statement, references, training & certification, and employment history.
4. **Preview and Submit**: Review your application before the final submission. Once ready, submit your application online.

All applications must be submitted online; paper applications or applications sent via email will not be accepted.

Source:
• From pages 1-5 of the Technical Documentation for UNDP’s eRecruit System (link).


## Exercise 2

Talk to your assistant via the API

https://platform.openai.com/docs/assistants/overview

In [5]:
# Create a thread and attach the file to the message
thread = client.beta.threads.create(
  messages=[
    {
      "role": "user",
      "content": "what is my name?",
    }
  ]
)

run = client.beta.threads.runs.create_and_poll(
    thread_id=thread.id, assistant_id=assistant.id
)

messages = list(client.beta.threads.messages.list(thread_id=thread.id, run_id=run.id))

message_content = messages[0].content[0].text
print(message_content.value)

I'm sorry, but I don't have access to your personal information, including your name. If you provide me with your name or any other details you'd like to share, I can assist you further!


## Exercise 3

Create an assistant that will call a weather API, given the user's answer and return the proper answer.

See the documentation of the weather API here: https://open-meteo.com/en/docs

In [74]:
import requests

def get_weather_forecast(latitude, longitude):
    base_url = "https://api.open-meteo.com/v1/forecast"
    params = {
        "latitude": latitude,
        "longitude": longitude,
        "hourly": "temperature_2m"
    }
    response = requests.get(base_url, params=params)
    return response.json()

# Example usage:
forecast = get_weather_forecast(52.52, 13.41)
display(forecast)



{'latitude': 52.52,
 'longitude': 13.419998,
 'generationtime_ms': 0.05996227264404297,
 'utc_offset_seconds': 0,
 'timezone': 'GMT',
 'timezone_abbreviation': 'GMT',
 'elevation': 38.0,
 'hourly_units': {'time': 'iso8601', 'temperature_2m': '°C'},
 'hourly': {'time': ['2024-10-29T00:00',
   '2024-10-29T01:00',
   '2024-10-29T02:00',
   '2024-10-29T03:00',
   '2024-10-29T04:00',
   '2024-10-29T05:00',
   '2024-10-29T06:00',
   '2024-10-29T07:00',
   '2024-10-29T08:00',
   '2024-10-29T09:00',
   '2024-10-29T10:00',
   '2024-10-29T11:00',
   '2024-10-29T12:00',
   '2024-10-29T13:00',
   '2024-10-29T14:00',
   '2024-10-29T15:00',
   '2024-10-29T16:00',
   '2024-10-29T17:00',
   '2024-10-29T18:00',
   '2024-10-29T19:00',
   '2024-10-29T20:00',
   '2024-10-29T21:00',
   '2024-10-29T22:00',
   '2024-10-29T23:00',
   '2024-10-30T00:00',
   '2024-10-30T01:00',
   '2024-10-30T02:00',
   '2024-10-30T03:00',
   '2024-10-30T04:00',
   '2024-10-30T05:00',
   '2024-10-30T06:00',
   '2024-10-30T07:00

{'city': 'Madrid', 'coordinates': {'latitude': 40.4168, 'longitude': -3.7038}}
40.4168 -3.7038


In [66]:
user_input = input('enter the city, I will tell you the weather')

In [67]:
prompt_weather= f"""
    #Question: {user_input}
    #Role
    You are a helpful assistant for users asking about the weather.

    #Context
    You are using an API (open-meteo), which expects the longitude and latitude, in order to return the weather.
    
    #Chain of thought
    Here are the steps you need to follow to complete this task:

        1. 'First, you need to retrieve the longitud of the city in {user_input}'
        2. 'Next, return the information in json.object format'

    #Example:
    This is an example. 
        -Question: 'What is the weather in Madrid?. 
        -Asnwer: 'city': 'Madrid', 'coordinates': 'latitude': 40.4168, 'longitude': -3.7038' in json format

    What are the subsequent steps they need to take to submit their application?"

    User Request: {user_input}
    """

In [76]:
def weather_assistant(prompt):
    message = client.beta.threads.messages.create(
        thread_id=thread.id,
        role="user",
        content=prompt
        )

    run = client.beta.threads.runs.create_and_poll(
        thread_id=thread.id, assistant_id='asst_Ii10AeQ9FGu5Rwfexd4PWb0l'
    ) 

    messages = list(client.beta.threads.messages.list(thread_id=thread.id, run_id=run.id))
    message_json = json.loads(message_content.value)

    coordinates = message_json['coordinates']
    return get_weather_forecast(coordinates['latitude'], coordinates['longitude'])

weather_assistant(prompt=prompt_weather)



{'latitude': 40.4375,
 'longitude': -3.6875,
 'generationtime_ms': 0.04303455352783203,
 'utc_offset_seconds': 0,
 'timezone': 'GMT',
 'timezone_abbreviation': 'GMT',
 'elevation': 666.0,
 'hourly_units': {'time': 'iso8601', 'temperature_2m': '°C'},
 'hourly': {'time': ['2024-10-29T00:00',
   '2024-10-29T01:00',
   '2024-10-29T02:00',
   '2024-10-29T03:00',
   '2024-10-29T04:00',
   '2024-10-29T05:00',
   '2024-10-29T06:00',
   '2024-10-29T07:00',
   '2024-10-29T08:00',
   '2024-10-29T09:00',
   '2024-10-29T10:00',
   '2024-10-29T11:00',
   '2024-10-29T12:00',
   '2024-10-29T13:00',
   '2024-10-29T14:00',
   '2024-10-29T15:00',
   '2024-10-29T16:00',
   '2024-10-29T17:00',
   '2024-10-29T18:00',
   '2024-10-29T19:00',
   '2024-10-29T20:00',
   '2024-10-29T21:00',
   '2024-10-29T22:00',
   '2024-10-29T23:00',
   '2024-10-30T00:00',
   '2024-10-30T01:00',
   '2024-10-30T02:00',
   '2024-10-30T03:00',
   '2024-10-30T04:00',
   '2024-10-30T05:00',
   '2024-10-30T06:00',
   '2024-10-30T07:0

### If you want to, there is a hint here:

OpenAI Chatbots / Assistants have a way to respond in json format. 

Explore the function calling functionality